In [0]:
# =====================================
# DISPARO DO JOB BRONZE
# =====================================
import requests
import json

In [0]:
# URL da API do Databricks Jobs
DATABRICKS_INSTANCE = "https://<sua-instancia>.azuredatabricks.net"
JOB_NAME = "Job_d_ingesta_Foco_Queim"

In [0]:
# Token de acesso (configure como secret no Databricks)
DATABRICKS_TOKEN = dbutils.secrets.get("scope_databricks", "token_jobs")

In [0]:
# Busca ID do job pelo nome
headers = {"Authorization": f"Bearer {DATABRICKS_TOKEN}"}
resp = requests.get(f"{DATABRICKS_INSTANCE}/api/2.1/jobs/list", headers=headers).json()
job_id = next((j["job_id"] for j in resp["jobs"] if j["settings"]["name"] == JOB_NAME), None)

if not job_id:
    raise ValueError(f"Job {JOB_NAME} não encontrado no workspace Databricks.")

In [0]:
# Executa o job passando data_ref_carga = data atual
body = {
    "job_id": job_id,
    "notebook_params": {
        "catalog": "datamasters",
        "schema_in": "b_inep",
        "table_in": "d_foco_queim_inc",
        "schema_out": "b_inep",
        "table_out": "d_foco_queim_inc",
        "data_ref_carga": datetime.now().strftime("%Y-%m-%d")
    }
}

resp_run = requests.post(f"{DATABRICKS_INSTANCE}/api/2.1/jobs/run-now",
                         headers=headers,
                         data=json.dumps(body))

print("Job Bronze disparado com sucesso 🚀")
print(resp_run.json())